<a href="https://colab.research.google.com/github/juelha/IANNWTF/blob/sabine/HW06_Sabine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf

#1 Data set
We will work with the Tensorflow Cifar10 dataset. https://www.cs.toronto.edu%7Ekriz/cifar.html. It contains 60.000 coloured images of cells with equal sizes. Each
image corresponds to one of 10 categories. The dataset is already implemented as a
keras.dataset module and very convenient to load!
Perform necessary or beneficial preprocessing steps.1 2

In [197]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

assert train_images.shape == (50000, 32, 32, 3)
assert test_images.shape == (10000, 32, 32, 3)
assert train_labels.shape == (50000, 1)
assert test_labels.shape == (10000, 1)

# TO DO: use min-max feature here or z scores 
train_images = train_images /  255.0
test_images = test_images / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

#print(train_dataset.take(1))

def preprocessing(tensor):
  """ apply a preprocessing pipeline to the given dataset
  :param tensor: data to be preprocessed
  :return: preprocessed dataset
  """
  # map the labels to one_hot labels and expand the last dimension of the images 
  tensor = tensor.map(lambda images, labels: (images, tf.one_hot(labels, 10)))
  #print(tensor.take(1))
  # cache this progress in memory
  tensor = tensor.cache()
  # shuffle, batch, prefetch
  tensor = tensor.shuffle(1000)
  tensor = tensor.batch(64)
  tensor = tensor.prefetch(20)
  # return preprocessed dataset
  return tensor

# apply the preprocessing pipeline to both the training and the test dataset
train_dataset = train_dataset.apply(preprocessing)
test_dataset = test_dataset.apply(preprocessing)

for image, label in train_dataset: 
  image_shape = list(image.shape[1:])
  image_shape.insert(0, 1)
  break

print(image_shape)


[1, 32, 32, 3]


## 2.1 ResNet
• Start with implementing a callable ResidualBlock class. 3
Implement an init function defining the main building blocks: Your residual
block should consist of multiple alterations of Convolution and Batch Normalization layers. You will need to make sure that the output of the block has the same
dimensions as its input. 4

Implement a call function. 5

• Implement a callable ResNet class, consisting of a convolutional layer followed by
multiple Residual Blocks and an output layer. 6
To further explore your networks behaviour, it might be convenient to implement
it in a way that you can easily alter the number of Residual Blocks. 7
Implement the network’s call function. 8


In [193]:
# Custom ResidualBlock Model
class ResidualBlock(tf.keras.Model):
    
    def __init__(self, mode = "normal", input_shape = (32,32,3), out_filters = 64):
        super(ResidualBlock, self).__init__()

        self.batch_normal = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.Activation(tf.nn.relu)
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")

        self.list_layers = []
        self.transform_original = []

        if mode == "normal":
          self.list_layers.append(tf.keras.layers.Conv2D(filters=out_filters/2, kernel_size =(3,3), padding="same"))
          self.list_layers.append(tf.keras.layers.Conv2D(filters=out_filters, kernel_size =(3,3), padding="same"))

          # transform original input to also have 256 channels
          self.transform_original.append(tf.keras.layers.Conv2D(filters=out_filters, kernel_size=(1,1)))
    
        # some blocks in ResNetV2 have a MaxPool with 1x1 pool size and strides of 2 instead
        elif mode == "strided":
           
            # do strided convolution (reducing feature map size)
            self.list_layers.append(tf.keras.layers.Conv2D(filters=out_filters/2, kernel_size =(3,3), padding="same"))
            self.list_layers.append(tf.keras.layers.Conv2D(filters=out_filters, kernel_size =(3,3), padding="same", strides=(2,2)))

            # transform original input with 1x1 strided max pooling to match output shape
            self.transform_original.append(tf.keras.layers.Conv2D(filters=out_filters, kernel_size=(1,1)))
            self.transform_original.append(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
            
        # other ResNetV2 blocks keep both the size and channel number constant
        elif mode == "constant":
            self.list_layers.append(tf.keras.layers.Conv2D(filters=out_filters, kernel_size =(3,3), padding="same"))
            self.transform_original.append(tf.keras.layers.Conv2D(filters=out_filters, kernel_size =(1,1), kernel_initializer="Ones", padding="same", trainable = False ))


    def call(self, x_in):

        # have an initial Conv layer before the first res block (increasing the n of channels)
        x = self.conv1(x_in) 
        x_out = x
        for layer in self.list_layers:
          x_out = layer(x_out)

        for layer in self.transform_original:
          x = layer(x)
          print(x_out.shape)

        x_out = self.batch_normal(x_out)
        x_out = self.activation(x_out)

        x_out = tf.keras.layers.Add()([x_out, x])
        return x_out


class MyModel(tf.keras.Model): 
  def __init__(self):
    super(MyModel, self).__init__()

    self.block1 = ResidualBlock(mode = 'strided', input_shape = image_shape)
    self.block2 = ResidualBlock(mode = 'normal', input_shape = image_shape)
  
  def call(self, x):
    x_out = self.block1(x)
    x_out = self.block2(x_out)

    return x_out 



In [198]:
#image_shape = (1, 32,32,3)
dummy = tf.ones(image_shape)

#resblock_try = ResidualBlock(mode = 'strided', input_shape = (32,32,3))
#out = resblock_try(dummy)

model = MyModel()

output = model(dummy)





(1, 16, 16, 64)
(1, 16, 16, 64)
(1, 16, 16, 64)


## 2.2 DenseNet
• Start with the Implementation of a callable class for your Transition Layers.
9
• Implement the Dense Block class. Remember, a Dense Block is a series of convolutions where the original input is concatenated with the output of the convolution
operation. 10 You may want to implement a subclass Block containing the convolution layer 11 and the concatenation implemented in the call function.
Implement your Dense Block class in a way that you can easily alter the number
of Dense Blocks you want to use. 12

• Implement the DenseNet class.
– You will want to have a set of layers before your Dense Blocks fixing the
desired channel dimension you need to pass on to the Dense Blocks.
– You will want to consecutively alter Dense Blocks and Transition Layers. 13
14 Remember the use of the growth rate when creating the Transition Layer
instances. 15
– After the last Dense Block make sure to include a proper set of output layers.
16
Implement the respective call function. 17

In [66]:
class TransitionLayer():

  def __init__(self, input_shape):
    self.batch_normal = tf.keras.layers.BatchNormalization(epsilon=1.001e-05)
    self.activation = tf.keras.layers.Activation(tf.nn.relu)
  
    self.conv = tf.keras.layers.Conv2D(filters = reduce_filters_to, kernel_size=(1,1), padding="valid", use_bias=False)

    # bottleneck, reducing the number of feature maps
    #(floor divide current number of filters by two for the bottleneck)
    self.bottleneck = input_shape//2
    # reduce the height and width of the feature map (not too useful for low-res input)
    
    self.pool = tf.keras.layers.AvgPool2D(pool_size=(2,2), strides = (2,2), padding = 'valid')
  
  def call(self, x):
    x = self.batch_normal(x)
    x = self.activation(x)
    x = self.conv(x)
    x = self.bottleneck(x)

    return x


In [65]:
class DenseBlock(): 

  def __init__(self): 
    super(DenseBlock, self).__init__()
    self.batch_normal =  tf.keras.layers.BatchNormalization(epsilon=1.001e-05)
    self.activation = tf.keras.layers.Activation(tf.nn.relu)
    
    # 1x1 convolution with 128 filters (padding "valid" because with 1x1 we don't need padding)
    self.conv1 = tf.keras.layers.Conv2D(n_filters, kernel_size=(1,1), padding="valid", use_bias=False)
    # 3x3 convolution with 32 filters (to be concatenated with the input)
    self.conv2 = tf.keras.layers.Conv2D(new_channels, kernel_size=(3,3), padding="same", use_bias=False)

  
  def block(self, x):
    x_out = self.batch_normal(x) 
    x_out = self.activation(x_out)
    x_out = self.conv1(x_out)
    x_out = self.batch_normal(x_out)
    x_out = self.activation(x_out)
    x_out = self.conv2(x_out)
    return x_out
  
  def call(self, x):
    # Concatenate layer (just a tf.keras.layers.Layer that calls tf.concat)
    x_out = tf.keras.layers.Concatenate(axis=3)([x, block(x)]) # axis 3 for channel dimension
    return x_out


# Custom Layer
class DenseNet(tf.keras.layers.Layer):

    def __init__(self, units=8):
        super(DenseNet, self).__init__()
        self.units = units
        self.activation = tf.nn.softmax

    def build(self, input_shape): 
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

    def call(self, inputs): 
        x = tf.matmul(inputs, self.w) + self.b
        x = self.activation(x)
        return x